# Data setup for qualitative evaluation

## Setup

### Modules

In [1]:
from pathlib import Path

import nibabel as nib
import nrrd
import numpy as np

import tractseg.config as config
import tractseg.libs.data_utils
import tractseg.utils.crop

### Data

In [2]:
SUBJECT = "987983"
EXPERIMENTS = ["peaks", "fodfs", "rank_3_approx"]

PATH_REL_DATA = "data/HCP"
PATH_REL_DIFFUSION = "Diffusion"
PATH_REL_SEGMENTATIONS = "segmentations"
PATH_REL_REFERENCE = "bundle_masks"
FILENAME_MASK_CROPPED = "mask_cropped.nii.gz"
FILENAME_MASK = "nodif_brain_mask.nii.gz"
FILENAME_REFERENCE = "bundle_masks.nii.gz"

path_dir_subject = Path(config.PATH_CWD) / PATH_REL_DATA / SUBJECT
path_dir_experiments = Path(config.PATH_DIR_EXP)
paths_segmentations = [path_dir_experiments / experiment / PATH_REL_SEGMENTATIONS / f"{SUBJECT}_segmentation.nii.gz" for experiment in EXPERIMENTS]
path_reference = path_dir_subject / PATH_REL_REFERENCE / FILENAME_REFERENCE
path_mask_cropped = path_dir_subject / PATH_REL_DIFFUSION / FILENAME_MASK_CROPPED
path_mask = path_dir_subject / PATH_REL_DIFFUSION / FILENAME_MASK

### Utilities

In [3]:
def load_nrrd(path):
    img, header_img = nrrd.read(path)
    return img, header_img


def load_nifti(path):
    img_nifti = nib.load(path)
    img = img_nifti.get_fdata()
    affine_img = img_nifti.affine
    return img, affine_img


def load_img(path):
    header_img, affine_img = None, None
    if path.suffixes == [".nrrd"]:
        img, header_img = load_nrrd(path)
    elif path.suffixes == [".nii", ".gz"]:
        img, affine_img = load_nifti(path)
    else:
        raise ValueError("Unsupported input file type.")

    return img, header_img, affine_img


def save_img(path, img, img_header=None, affine=None):
    if path.suffixes == [".nrrd"]:
        nrrd.write(str(path), img, img_header)
    elif path.suffixes == [".nii", ".gz"]:
        img_output = nib.Nifti1Image(img, affine if affine is not None else np.eye(4))
        nib.save(img_output, str(path))
    else:
        raise ValueError("Unsupported file type.")

## Map segmentations back to HCP data space

In [4]:
mask, _, affine = load_img(path_mask)
bb = tractseg.utils.crop.bounding_box(mask)

display(bb)

array([[ 20, 125],
       [ 23, 156],
       [  9, 112]])

In [5]:
mask_cropped, _, _ = load_img(path_mask_cropped)
mask_square, transformation = tractseg.libs.data_utils.pad_and_scale_img_to_square_img(mask_cropped, target_size=144, nr_cpus=1)

display(mask_square.shape)
display(transformation)

(144, 144, 144)

{'original_shape': (105, 133, 103),
 'pad_x': 14.0,
 'pad_y': 0.0,
 'pad_z': 15.0,
 'zoom': 1.0827067669172932}

In [6]:
reference, _, _ = load_img(path_reference)
reference = reference.astype(np.int16)

In [7]:
# for path_segmentations in paths_segmentations:
#     segmentation, _, _ = load_img(path_segmentations)
#     segmentation = segmentation.astype(np.int16)
#     segmentation = tractseg.libs.data_utils.cut_and_scale_img_back_to_original_img(segmentation, transformation, nr_cpus=1)

#     segmentation_rescaled = np.zeros((145, 174, 145, 72), dtype=np.int16)
#     segmentation_rescaled[bb[0, 0] : bb[0, 1], bb[1, 0] : bb[1, 1], bb[2, 0] : bb[2, 1], :] = segmentation

#     path_segmentations_rescaled = path_segmentations.parents[0] / f"{SUBJECT}_segmentation_rescaled.nii.gz"
#     save_img(path_segmentations_rescaled, segmentation_rescaled, affine=affine)

#     path_segmentations_rescaled_fp = path_segmentations.parents[0] / f"{SUBJECT}_segmentation_rescaled_fp.nii.gz"
#     save_img(path_segmentations_rescaled_fp, np.clip(segmentation_rescaled - reference, a_min=0, a_max=None), affine=affine)

#     path_segmentations_rescaled_fn = path_segmentations.parents[0] / f"{SUBJECT}_segmentation_rescaled_fn.nii.gz"
#     save_img(path_segmentations_rescaled_fn, np.clip(reference - segmentation_rescaled, a_min=0, a_max=None), affine=affine)

In [10]:
tracts = np.array([25, 15, 4])
for path_segmentations in paths_segmentations:
    segmentation, _, _ = load_img(path_segmentations)
    segmentation = segmentation.astype(np.int16)
    segmentation = tractseg.libs.data_utils.cut_and_scale_img_back_to_original_img(segmentation, transformation, nr_cpus=1)

    segmentation_rescaled = np.zeros((145, 174, 145, 72), dtype=np.int16)
    segmentation_rescaled[bb[0, 0] : bb[0, 1], bb[1, 0] : bb[1, 1], bb[2, 0] : bb[2, 1], :] = segmentation

    for tract in tracts:
        path_segmentations_rescaled = path_segmentations.parents[0] / f"{SUBJECT}_segmentation_rescaled_{tract}.nii.gz"
        save_img(path_segmentations_rescaled, segmentation_rescaled[..., tract], affine=affine)

        path_segmentations_rescaled_fp = path_segmentations.parents[0] / f"{SUBJECT}_segmentation_rescaled_fp_{tract}.nii.gz"
        save_img(
            path_segmentations_rescaled_fp, np.clip(segmentation_rescaled[..., tract] - reference[..., tract], a_min=0, a_max=None), affine=affine
        )

        path_segmentations_rescaled_fn = path_segmentations.parents[0] / f"{SUBJECT}_segmentation_rescaled_fn_{tract}.nii.gz"
        save_img(
            path_segmentations_rescaled_fn, np.clip(reference[..., tract] - segmentation_rescaled[..., tract], a_min=0, a_max=None), affine=affine
        )